In [63]:
import requests
from bs4 import BeautifulSoup

In [64]:
# h2 = naglowki
# p = tekst (notatki?)
# ul = pole, ktore wyglada jak lista, a elementy są w kropce
# li = to są elementy na liscie
# dl = cale pole, ktore zawiera dymki NPC

url = "https://warcraft.wiki.gg/wiki/Champions_of_Quel%27Danas"
odpowiedz = requests.get(
    url,
    headers={"User-Agent": "WoW_PolishTranslationProject -> contact: py.piotrx@gmail.com"},
    timeout=30
)
html = odpowiedz.text

soup = BeautifulSoup(html, "html.parser")

In [ ]:
ODDZIELONY_SEP = "---------"

tresc = soup.select_one("#bodyContent")

pods_misji = tresc.find(class_="infobox darktable questbox")
cele_misji = tresc.find(id="Objectives")
opis_misji = tresc.find(id="Description")

pods_misji_slownik = {}
if pods_misji:
    print("==========PODSUMOWANIE MISJI==========")
    tytul_misji   = pods_misji.find(class_="above-header")
    tytul_misji_t = tytul_misji.get_text()

    pods_misji_slownik["Tytuł"] = tytul_misji_t

    for ps in pods_misji.find_all("th"):
        naglowki_t = ps.get_text().strip()
        zawartosc = ps.find_next_sibling()
        if not zawartosc or zawartosc.name != "td":
            continue

        match naglowki_t:
            case "Start":      naglowki_t = "Start_NPC"
            case "End":        naglowki_t = "Koniec_NPC"
            case "Level":      naglowki_t = "Wymagany_Poziom"
            case "Experience": naglowki_t = "Doświadczenie"
            case "Rewards":    naglowki_t = "Nagrody"
            case "Previous":   naglowki_t = "Poprzednia_Misja"
            case "Next":       naglowki_t = "Następna_Misja"

        pods_misji_slownik[naglowki_t] = zawartosc.get_text().strip()
        
    print(pods_misji_slownik)
print("\n")

cele_misji_slownik = { "Główny": {}, "Podrzędny": {} }
if cele_misji:
    print("==========CELE MISJI==========")
    naglowki     = cele_misji.find_parent("h2")

    glowny_cel = naglowki.find_next_sibling("p")
    if glowny_cel:
        cele_misji_slownik["Główny"][1] = glowny_cel.get_text().strip()


    lista_celi   = glowny_cel.find_next_sibling("ul")
    if lista_celi:
        podcele = lista_celi.find_all("li")
        for i, pc in enumerate(podcele, start=1):
            podrzedny_cel_t = pc.get_text().strip()
            cele_misji_slownik["Podrzędny"][i] = podrzedny_cel_t
    print(cele_misji_slownik)
print("\n")

opis_misji_slownik = {}
if opis_misji:
    print("==========OPIS MISJI==========")
    opis = opis_misji.find_parent("h2")

    licznik = 0
    while True:
        opis = opis.find_next_sibling()
        if opis is None or opis.name == "h2":
            break
        elif opis.name == "p":
            licznik += 1
            opis_misji_slownik[licznik] = opis.get_text().strip()
    print(opis_misji_slownik)
print("\n")




==========PODSUMOWANIE MISJI==========
{'Tytuł': "Champions of Quel'Danas", 'Start_NPC': 'Lady Liadrin', 'Koniec_NPC': "Lor'themar Theron", 'Wymagany_Poziom': '80', 'Doświadczenie': '1,750', 'Nagrody': '23 40', 'Poprzednia_Misja': '[80]\xa0Last Bastion of the Light', 'Następna_Misja': '[80]\xa0The Hour of Need & \xa0[80]\xa0A Safe Path'}


==========CELE MISJI==========
{'Główny': {1: "Rally Faerin Lothar, Lothraxion, and Alonsus Faol at Light's Spire."}, 'Podrzędny': {1: 'Recruit Faerin Lothar', 2: 'Recruit Alonsus Faol', 3: 'Recruit Lothraxion', 4: "Meet Lor'themar at Sunstrider Rise."}}


==========OPIS MISJI==========
{1: "Lor'themar has formed a rally point at the Light's Spire, while the civilians there are evacuated to Silvermoon. But the forces of the Vanguard of Light are scattered throughout the battlefield.", 2: "We need to fight as one to defeat this host. Together we can defeat Xal'atath and save the Sunwell."}


